In [54]:
import os
import pandas as pd
import numpy as np

In [45]:
mojo_2014=pd.read_csv("Box Office Mojo 2014.csv")
mojo_2015=pd.read_csv("Box Office Mojo 2015.csv")
mojo_2016=pd.read_csv("Box Office Mojo 2016.csv")
mojo_2017=pd.read_csv("Box Office Mojo 2017.csv")
mojo_2018=pd.read_csv("Box Office Mojo 2018.csv")

mojo_df=pd.concat([mojo_2014, mojo_2015, mojo_2016, mojo_2017, mojo_2018])
mojo_df=mojo_df.rename(columns={"Movie Title": "Title"})
mojo_df=mojo_df[["Title", "Total Gross", "Release Date", "Production Company", "Year"]]
mojo_df.head()

,Title,Total Gross,Release Date,Production Company,Year
0,Guardians of the Galaxy,"$333,176,600",Aug 1,Walt Disney Studios Motion Pictures\n\n,2014
1,The Hunger Games: Mockingjay - Part 1,"$337,135,885",Nov 21,Lionsgate\n\n,2014
2,Captain America: The Winter Soldier,"$259,766,572",Apr 4,Walt Disney Studios Motion Pictures\n\n,2014
3,The Lego Movie,"$257,760,692",Feb 7,Warner Bros.\n\n,2014
4,Transformers: Age of Extinction,"$245,439,076",Jun 27,Paramount Pictures\n\n,2014


In [46]:
movie_collect=pd.read_csv("Movie Collection Data.csv")
movie_collect=movie_collect[["title", "genres", "budget", "revenue", "runtime"]]
movie_collect=movie_collect.rename(columns={"title": "Title", "genres": "Genre", "budget": "Budget", "revenue": "Revenue", "runtime": "Runtime"})
movie_collect.head()

,Title,Genre,Budget,Revenue,Runtime
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",30000000,373554033.0,81.0
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",65000000,262797249.0,104.0
2,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",0,0.0,101.0
3,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",16000000,81452156.0,127.0
4,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",0,76578911.0,106.0


In [71]:
merged_df=pd.merge(mojo_df, movie_collect, on="Title", how= "left")
merged_df=merged_df.dropna()
merged_df=merged_df.loc[merged_df["Budget"] != "0"]
merged_df=merged_df.set_index("Title")

merged_df.head(20)

,Total Gross,Release Date,Production Company,Year,Genre,Budget,Revenue,Runtime
Title,,,,,,,,
Guardians of the Galaxy,"$333,176,600",Aug 1,Walt Disney Studios Motion Pictures\n\n,2014,"[{'id': 28, 'name': 'Action'}, {'id': 878, 'na...",170000000,7.733286e+08,121.0
The Hunger Games: Mockingjay - Part 1,"$337,135,885",Nov 21,Lionsgate\n\n,2014,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",125000000,7.521002e+08,123.0
Captain America: The Winter Soldier,"$259,766,572",Apr 4,Walt Disney Studios Motion Pictures\n\n,2014,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",170000000,7.147666e+08,136.0
The Lego Movie,"$257,760,692",Feb 7,Warner Bros.\n\n,2014,"[{'id': 12, 'name': 'Adventure'}, {'id': 16, '...",60000000,4.691607e+08,100.0
Transformers: Age of Extinction,"$245,439,076",Jun 27,Paramount Pictures\n\n,2014,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",210000000,1.091405e+09,165.0
Maleficent,"$241,410,378",May 30,Walt Disney Studios Motion Pictures\n\n,2014,"[{'id': 14, 'name': 'Fantasy'}, {'id': 12, 'na...",180000000,7.585398e+08,97.0
X-Men: Days of Future Past,"$233,921,534",May 23,Twentieth Century Fox\n\n,2014,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",250000000,7.478628e+08,131.0
Dawn of the Planet of the Apes,"$208,545,589",Jul 11,Twentieth Century Fox\n\n,2014,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",170000000,7.106446e+08,130.0
Big Hero 6,"$222,527,828",Nov 7,Walt Disney Studios Motion Pictures\n\n,2014,"[{'id': 12, 'name': 'Adventure'}, {'id': 10751...",165000000,6.521054e+08,102.0
